In [ ]:
!pip install torchtext==0.6.0 --quiet
!python -m spacy download de --quiet

     |████████████████████████████████| 71kB 5.2MB/s 
     |████████████████████████████████| 1.1MB 8.5MB/s 
     |████████████████████████████████| 14.9MB 534kB/s 
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
import spacy
import random
from tqdm import tqdm
from torchtext.data.metrics import bleu_score


class Encoder(nn.Module):
    def __init__(self, enc_input_size, enc_embed_size, enc_hidden_size, num_layers, dec_hidden_size, dropout):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(enc_input_size, enc_embed_size)
        self.dropout = nn.Dropout(p=dropout)
        self.rnn = nn.GRU(enc_embed_size, enc_hidden_size, num_layers, bidirectional=True)
        self.fc = nn.Linear(enc_hidden_size*2, dec_hidden_size)

    def forward(self, x):
        embeddings = self.embedding(x)
        embeddings_d = self.dropout(embeddings)

        output_states, hidden = self.rnn(embeddings_d)

        # hidden is stacked [forward 1, backward 1, f2, b2,........forward_last, backward_last]
        forward_last = hidden[0:1, :, :]
        backward_last = hidden[1:2, :, :]

        combined_last_hidden = torch.cat((forward_last, backward_last), dim=2)

        last_hidden_to_be_fed_to_decoder = self.fc(combined_last_hidden)

        return output_states, last_hidden_to_be_fed_to_decoder


class Decoder(nn.Module):
    def __init__(self, dec_input_size, dec_embed_size, enc_hidden_size, dec_hidden_size, num_layers, target_size, dropout):
        super(Decoder, self).__init__()
        self.target_size = target_size
        self.embedding = nn.Embedding(dec_input_size, dec_embed_size)
        self.dropout = nn.Dropout(p=dropout)
        self.energy = nn.Linear(enc_hidden_size * 2 + dec_hidden_size, 1)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=0)
        self.rnn = nn.GRU(enc_hidden_size*2 + dec_embed_size, dec_hidden_size, num_layers)
        self.fc = nn.Linear(dec_hidden_size, target_size)

    def forward(self, dec_input, dec_hidden_state, encoder_states):
        src_seq_length = encoder_states.shape[0]

        dec_input = dec_input.unsqueeze(0)
        dec_embeddings = self.embedding(dec_input)
        dec_embeddings_d = self.dropout(dec_embeddings)

        dec_hidden_state_reshaped = dec_hidden_state.repeat(src_seq_length, 1, 1)

        energy = self.relu(self.energy(torch.cat((dec_hidden_state_reshaped, encoder_states), dim=2)))

        attention_weights = self.softmax(energy)

        context_vector = torch.einsum("snk,snl->knl", attention_weights, encoder_states)

        rnn_input = torch.cat((context_vector, dec_embeddings_d), dim=2)

        dec_output, dec_next_hidden_state = self.rnn(rnn_input, dec_hidden_state)

        predictions = self.fc(dec_output).squeeze(0)

        return predictions, dec_next_hidden_state


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_force_ratio=0.5):
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.target_size

        out_states, hidden = self.encoder(src)

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        x = trg[0, :]

        for t in range(1, trg_len):
            out, hidden = self.decoder(x, hidden, out_states)
            best_guess = out.argmax(1)

            outputs[t] = out

            x = trg[t] if random.random() < teacher_force_ratio else best_guess

        return outputs


def train(model, iterator, optimizer, criterion, clip, device):
    model.train()
    epoch_loss = 0

    for _, batch in tqdm(enumerate(iterator), total=len(iterator)):

        inp = batch.src.to(device)
        trg = batch.trg.to(device)

        outs = model(inp, trg)
        outs = outs[1:].reshape(-1, outs.shape[2])

        trg = trg[1:].reshape(-1)

        optimizer.zero_grad()

        loss = criterion(outs, trg)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model, iterator, criterion, device):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():

        for _, batch in tqdm(enumerate(iterator), total=len(iterator)):
            inp_data = batch.src.to(device)
            target = batch.trg.to(device)

            output = model(inp_data, target, teacher_force_ratio= 0)  # turn off teacher force

            output = output[1:].reshape(-1, output.shape[2])
            target = target[1:].reshape(-1)

            loss = criterion(output, target)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def translate_sentence(model, sentence, SRC, TRG, device, max_length=50):
    spacy_ger = spacy.load("de")

    # Create tokens using spacy and everything in lower case (which is what our vocab is)
    if type(sentence) == str:
        tokens = [token.text.lower() for token in spacy_ger(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # Add <SOS> and <EOS> in beginning and end respectively
    tokens.insert(0, SRC.init_token)
    tokens.append(SRC.eos_token)

    # Go through each german token and convert to an index
    text_to_indices = [SRC.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        out_states, hidden = model.encoder(sentence_tensor)

    outputs = [TRG.vocab.stoi["<sos>"]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden = model.decoder(previous_word, hidden, out_states)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == TRG.vocab.stoi["<eos>"]:
            break

    translated_sentence = [TRG.vocab.itos[idx] for idx in outputs]

    # remove start token
    return translated_sentence[1:]


def bleu(data, model, SRC, TRG, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["src"]
        trg = vars(example)["trg"]

        prediction = translate_sentence(model, src, SRC, TRG, device)
        prediction = prediction[:-1]  # remove <eos> token
        outputs.append(prediction)

        targets.append([trg])

    return bleu_score(outputs, targets)


if __name__ == "__main__":

    SRC_ger = Field(tokenize='spacy', tokenizer_language='de', lower=True, init_token="<sos>", eos_token="<eos>")
    TRG_eng = Field(tokenize='spacy', tokenizer_language='en', lower=True, init_token="<sos>", eos_token="<eos>")

    train_data, valid_data, test_data = Multi30k.splits(exts=(".de", ".en"), fields=(SRC_ger, TRG_eng))

    SRC_ger.build_vocab(train_data, max_size=10000, min_freq=2)
    TRG_eng.build_vocab(train_data, max_size=10000, min_freq=2)

    # print(SRC_ger.vocab.stoi), print(TRG_eng.vocab.stoi)

    num_epochs = 100
    learning_rate = 0.001
    batch_size = 128
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print('device:',device)
    input_size_encoder = len(SRC_ger.vocab)
    input_size_decoder = len(TRG_eng.vocab)
    output_size = len(TRG_eng.vocab)
    encoder_embedding_size = 300
    decoder_embedding_size = 300
    enc_hidden_size = dec_hidden_size = 1024  # Needs to be the same for both encoder and decoder RNNs
    num_layers = 1
    enc_dropout = 0.5
    dec_dropout = 0.5
    sort_key = lambda x: len(x.src)
    pad_idx = TRG_eng.vocab.stoi["<pad>"]
    sentence = "ein boot mit mehreren männern darauf wird von einem großen pferdegespann ans ufer gezogen."

    train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
        (train_data, valid_data, test_data),
        batch_size=batch_size,
        sort_within_batch=True,
        sort_key=sort_key,
        device=device,
    )

    encoder_net = Encoder(input_size_encoder, encoder_embedding_size, enc_hidden_size,
                          num_layers, dec_hidden_size, enc_dropout).to(device)

    decoder_net = Decoder(input_size_decoder, decoder_embedding_size, enc_hidden_size, dec_hidden_size,
                          num_layers, output_size, dec_dropout).to(device)

    model = Seq2Seq(encoder_net, decoder_net, device).to(device)

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
    
    print('Input German sentence:', sentence)

    for epoch in range(num_epochs):
        print(f"------------------[Epoch {epoch + 1} / {num_epochs}]-------------------------")

        translated_sentence = translate_sentence(model, sentence, SRC_ger, TRG_eng, device, max_length=50)
        print('\nTranslated Sentence:', ' '.join(translated_sentence))

        train_loss = train(model, train_iterator, optimizer, criterion, clip=0.5, device=device)
        val_loss = evaluate(model, valid_iterator, criterion, device=device)

        print(f'Epoch train loss: {train_loss}, validation loss: {val_loss}')

        if (epoch + 1) % 19 == 0:
            save_path = f'/content/drive/My Drive/Jovian/mt/epoch{epoch + 1}.pth'
            torch.save(model, save_path)

    save_path = f'/content/drive/My Drive/Jovian/mt/final_attention.pth'
    torch.save(model, save_path)
    score = bleu(test_data[1:100], model, SRC_ger, TRG_eng, device)
    print(f"Bleu score: {score * 100}")

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:02<00:00, 601kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 170kB/s]


downloading mmt_task1_test2016.tar.gz


...

Epoch train loss: 1.4595771571087943, validation loss: 5.35418438911438
------------------[Epoch 100 / 100]-------------------------


  0%|          | 0/227 [00:00<?, ?it/s]


Translated Sentence: a boat boat rowing men is pulled by a large maroon shore by large shore . <eos>


100%|██████████| 8/8 [00:00<00:00, 14.50it/s]


Epoch train loss: 1.486231567576068, validation loss: 5.374042212963104
Bleu score: 21.702828189568592


In [1]:
import jovian
jovian.commit(project='machine-translation-with-attention')

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Please enter your API key ( from https://jovian.ai/ ):
API KEY: ········
[jovian] Creating a new project "gitrohitjain/machine-translation-with-attention"
[jovian] Uploading notebook..
[jovian] Capturing environment..


[jovian] Error: Failed to read Anaconda environment using command: "conda env export -n base --no-builds"


[jovian] Committed successfully! https://jovian.ai/gitrohitjain/machine-translation-with-attention


'https://jovian.ai/gitrohitjain/machine-translation-with-attention'